In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from unityagents import UnityEnvironment
from torch.utils.data import TensorDataset, DataLoader

# Actor Network
class Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_size=256):
        super(Actor, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )
        self.mu = nn.Linear(hidden_size, act_dim)
        self.log_std = nn.Parameter(torch.zeros(act_dim))
        
    def forward(self, x):
        x = self.net(x)
        mu = torch.tanh(self.mu(x))
        std = torch.exp(self.log_std).expand_as(mu)
        return mu, std

# Centralized Critic Network
class Critic(nn.Module):
    def __init__(self, state_dim, hidden_size=256):
        super(Critic, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
    def forward(self, x):
        return self.net(x)

# MAPPO Agent
class MAPPO:
    def __init__(self, obs_dim, act_dim, state_dim, num_agents,
                 gamma=0.99, clip_param=0.2, ppo_epochs=10, lr=3e-4,
                 batch_size=64, gae_lambda=0.95, entropy_coef=0.01):
        
        self.actors = nn.ModuleList([Actor(obs_dim, act_dim) for _ in range(num_agents)])
        self.critic = Critic(state_dim)
        self.optimizer = optim.Adam([
            {'params': self.actors.parameters()},
            {'params': self.critic.parameters()}
        ], lr=lr)
        
        self.gamma = gamma
        self.clip_param = clip_param
        self.ppo_epochs = ppo_epochs
        self.batch_size = batch_size
        self.gae_lambda = gae_lambda
        self.entropy_coef = entropy_coef
        self.num_agents = num_agents
        self.act_dim = act_dim

    def act(self, obs, agent_idx):
        with torch.no_grad():
            obs_tensor = torch.FloatTensor(obs)
            mu, std = self.actors[agent_idx](obs_tensor)
            dist = torch.distributions.Normal(mu, std)
            action = dist.sample()
            log_prob = dist.log_prob(action).sum(-1)
        return action.numpy(), log_prob.numpy()

    def update(self, buffer):
        # Convert buffer to tensors
        states = torch.FloatTensor(np.array(buffer.global_states))
        next_states = torch.FloatTensor(np.array(buffer.next_global_states))
        individual_obs = torch.FloatTensor(np.array(buffer.individual_obs))
        actions = torch.FloatTensor(np.array(buffer.actions))
        rewards = torch.FloatTensor(np.array(buffer.rewards))
        dones = torch.FloatTensor(np.array(buffer.dones))
        old_log_probs = torch.FloatTensor(np.array(buffer.log_probs))
        agent_indices = torch.LongTensor(np.array(buffer.agent_indices))

        # Calculate advantages using GAE
        with torch.no_grad():
            values = self.critic(states).squeeze()
            next_values = self.critic(next_states).squeeze()
            
            advantages = torch.zeros_like(rewards)
            last_advantage = 0
            for t in reversed(range(len(rewards))):
                mask = 1.0 - dones[t]
                delta = rewards[t] + self.gamma * next_values[t] * mask - values[t]
                advantages[t] = delta + self.gamma * self.gae_lambda * mask * last_advantage
                last_advantage = advantages[t]
            returns = advantages + values

        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # Create dataset
        dataset = TensorDataset(individual_obs, actions, old_log_probs, 
                               advantages, returns, states, agent_indices)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        # PPO update
        for _ in range(self.ppo_epochs):
            for batch in dataloader:
                ind_obs_b, actions_b, old_log_probs_b, adv_b, ret_b, states_b, agent_idx_b = batch
                
                policy_losses = []
                entropy_losses = []
                
                # Update each agent's policy
                for agent_id in range(self.num_agents):
                    mask = agent_idx_b == agent_id
                    if mask.sum() == 0:
                        continue
                    
                    # Agent-specific data
                    agent_obs = ind_obs_b[mask]
                    agent_actions = actions_b[mask]
                    agent_old_log_probs = old_log_probs_b[mask]
                    agent_adv = adv_b[mask]
                    
                    # Calculate new policy
                    mu, std = self.actors[agent_id](agent_obs)
                    dist = torch.distributions.Normal(mu, std)
                    new_log_probs = dist.log_prob(agent_actions).sum(-1)
                    entropy = dist.entropy().mean()
                    
                    # Policy loss
                    ratio = (new_log_probs - agent_old_log_probs).exp()
                    surr1 = ratio * agent_adv
                    surr2 = torch.clamp(ratio, 1.0 - self.clip_param, 
                                      1.0 + self.clip_param) * agent_adv
                    policy_loss = -torch.min(surr1, surr2).mean()
                    
                    policy_losses.append(policy_loss)
                    entropy_losses.append(entropy)

                # Combine losses
                policy_loss = torch.stack(policy_losses).mean() if policy_losses else 0.0
                entropy_loss = torch.stack(entropy_losses).mean() if entropy_losses else 0.0
                
                # Value loss
                values_pred = self.critic(states_b).squeeze()
                value_loss = F.mse_loss(values_pred, ret_b)
                
                # Total loss
                total_loss = policy_loss + value_loss - self.entropy_coef * entropy_loss
                
                # Optimize
                self.optimizer.zero_grad()
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.actors.parameters(), 0.5)
                torch.nn.utils.clip_grad_norm_(self.critic.parameters(), 0.5)
                self.optimizer.step()
        
        buffer.clear()


# Experience Replay Buffer
class ReplayBuffer:
    def __init__(self):
        self.global_states = []
        self.individual_obs = []
        self.actions = []
        self.rewards = []
        self.next_global_states = []
        self.dones = []
        self.log_probs = []
        self.agent_indices = []
        
    def add(self, global_state, individual_obs, action, reward, 
            next_global_state, done, log_prob, agent_idx):
        self.global_states.append(global_state)
        self.individual_obs.append(individual_obs)
        self.actions.append(action)
        self.rewards.append(reward)
        self.next_global_states.append(next_global_state)
        self.dones.append(done)
        self.log_probs.append(log_prob)
        self.agent_indices.append(agent_idx)
        
    def clear(self):
        self.global_states = []
        self.individual_obs = []
        self.actions = []
        self.rewards = []
        self.next_global_states = []
        self.dones = []
        self.log_probs = []
        self.agent_indices = []


In [2]:

# Training Loop
# Declare agent
import sys
# Add the subfolder to sys.path
sys.path.append('./tennis')
env = UnityEnvironment(file_name="tennis/Tennis.app")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Environment parameters
env_info = env.reset(train_mode=True)[brain_name]
num_agents = len(env_info.agents)
obs_dim = env_info.vector_observations.shape[1]
print("state size: ", obs_dim)
act_dim = brain.vector_action_space_size
print("action size: ", act_dim)
state_dim = obs_dim * num_agents

# Initialize MAPPO
mappo = MAPPO(obs_dim, act_dim, state_dim, num_agents)
buffer = ReplayBuffer()


Mono path[0] = '/Users/nuocheng/Desktop/Reinforcement_Learning/Reinforcement-Learning-Projects/tennis/Tennis.app/Contents/Resources/Data/Managed'
Mono config path = '/Users/nuocheng/Desktop/Reinforcement_Learning/Reinforcement-Learning-Projects/tennis/Tennis.app/Contents/MonoBleedingEdge/etc'


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


state size:  24
action size:  2


In [ ]:
import os
from collections import deque

# Training parameters
max_episodes = 30000
max_steps = 1000
update_freq = 2048
print_interval = 20          # Print stats every 10 episodes
save_interval = 100          # Save model every 100 episodes
solved_threshold = 2.0       # Consider solved when average score > 0.5

# Track rewards
episode_rewards = []
moving_avg = []
moving_var = []  # New variance tracking
best_avg_reward = -np.inf
reward_window = deque(maxlen=100)  # For rolling average

# Create directory for model saves
if not os.path.exists('models'):
    os.makedirs('models')

# Training loop
for episode in range(max_episodes):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    episode_reward = 0

    for step in range(max_steps):
        global_state = states.reshape(-1)
        actions = []
        log_probs = []

        # Get actions from all agents
        for agent_id in range(num_agents):
            action, log_prob = mappo.act(states[agent_id], agent_id)
            actions.append(action)
            log_probs.append(log_prob)

        # Environment step
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done

        # Store experiences
        team_reward = max(rewards)
        episode_reward += team_reward
        next_global_state = next_states.reshape(-1)

        for agent_id in range(num_agents):
            buffer.add(
                global_state=global_state,
                individual_obs=states[agent_id],
                action=actions[agent_id],
                reward=team_reward,
                next_global_state=next_global_state,
                done=dones[agent_id],
                log_prob=log_probs[agent_id],
                agent_idx=agent_id
            )

        states = next_states

        # Update when buffer is full
        if len(buffer.rewards) >= update_freq:
            mappo.update(buffer)

        if np.any(dones):
            break
    
    # Update reward tracking
    episode_rewards.append(episode_reward)
    reward_window.append(episode_reward)
    current_avg = np.mean(reward_window)
    current_var = np.var(list(reward_window))  # Calculate variance
    moving_avg.append(current_avg)
    moving_var.append(current_var)  # Store variance
    
    # Save best model
    if current_avg > best_avg_reward:
        best_avg_reward = current_avg
        # torch.save(mappo.actor.state_dict(), f'models/best_actor.pth')
        # torch.save(mappo.critic.state_dict(), f'models/best_critic.pth')
    
    # Print statistics
    if (episode + 1) % print_interval == 0:
        print(f"Episode {episode + 1}, Total Reward: {episode_reward:.2f}, Average Reward (Last 100): {current_avg:.2f}, Max Reward: {np.max(episode_rewards[-print_interval:]):.2f}, Min Reward: {np.min(episode_rewards[-print_interval:]):.2f}")

    # Early stopping if solved
    if current_avg >= solved_threshold:
        print(f"\nEnvironment solved in {episode + 1} episodes!")
        print(f"Average Reward: {current_avg:.2f}")
        break

env.close()

# Plotting the rewards (add this at the end)
import matplotlib.pyplot as plt
# Modified Plotting Section
plt.figure(figsize=(12, 6))

# Calculate standard deviation from variance for visualization
std_dev = np.sqrt(np.array(moving_var))

# Plot main reward curve and average
plt.plot(episode_rewards, alpha=0.3, color='blue', label='Episode Reward')
plt.plot(moving_avg, linewidth=2, color='darkblue', label='100-Episode Average')

# Plot variance as shaded area around the mean
plt.fill_between(range(len(moving_avg)), 
                 np.array(moving_avg) - np.array(moving_var),
                 np.array(moving_avg) + np.array(moving_var),
                 color='skyblue', alpha=0.3, label='Variance')

plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Training Progress with Reward Variance')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('reward_with_variance.png')
plt.show()

Episode 20, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.10, Min Reward: 0.00
Episode 40, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.10, Min Reward: 0.00
Episode 60, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.10, Min Reward: 0.00
Episode 80, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.10, Min Reward: 0.00
Episode 100, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.10, Min Reward: 0.00
Episode 120, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.20, Min Reward: 0.00
Episode 140, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.00, Min Reward: 0.00
Episode 160, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.10, Min Reward: 0.00
Episode 180, Total Reward: 0.00, Average Reward (Last 100): 0.01, Max Reward: 0.10, Min Reward: 0.00
Episode 200, Total Reward: 0.10, Average Reward (Last 100): 0.01, Max Reward: 0.10, Min Reward: